#Collaborative Filtering

In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/AdarshKumar712/Recommendation_Engine.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 2), reused 15 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/cloned-repo
LICENSE  MovieLen100k  MovieLen1M  README.md


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [0]:
Movies = pd.read_csv("./MovieLen100k/movies.csv")
Tags = pd.read_csv("./MovieLen100k/tags.csv")
Ratings = pd.read_csv("./MovieLen100k/ratings.csv")
Links = pd.read_csv("./MovieLen100k/links.csv")

In [4]:
Movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
Movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [6]:
np.max(Movies["movieId"])

193609

In [0]:
#map MovieId with Sequence
Id_Map = {}
Reverse_Id_Map = {}
for i in range (0,len(Movies["movieId"])):
  Id_Map[Movies["movieId"][i]] = i
  Reverse_Id_Map[i] = Movies["movieId"][i]

In [8]:
len(Id_Map)

9742

In [9]:
Tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
Tags.describe()

,userId,movieId,timestamp
count,3683.000000,3683.000000,3.683000e+03
mean,431.149335,27252.013576,1.320032e+09
std,158.472553,43490.558803,1.721025e+08
min,2.000000,1.000000,1.137179e+09
25%,424.000000,1262.500000,1.137521e+09
50%,474.000000,4454.000000,1.269833e+09
75%,477.000000,39263.000000,1.498457e+09
max,610.000000,193565.000000,1.537099e+09


In [0]:
Tags = Tags.drop("timestamp",axis = 1)

In [12]:
Ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
Ratings = Ratings.drop("timestamp",axis = 1)

In [14]:
Links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Simple Filtering Based on the Ratings given

In [0]:
Users = (Ratings.userId.unique().shape[0])
Total_Movies  = (Movies.movieId.unique().shape[0])

In [0]:
Rating_matrix = np.zeros((Users+1,Total_Movies))

In [0]:
for i in range(0,len(Ratings['userId'])):
  Rating_matrix[Ratings.iloc[i,0],Id_Map[Ratings.iloc[i,1]]] = Ratings.iloc[i,2]

In [18]:
Rating_matrix
#Row_zero is the buffer

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

###User-item based Colaboration Filtering

In [0]:
cnts = np.sum((Rating_matrix>0),axis = 1)

In [0]:
rating_matrix = np.zeros((Users+1,Total_Movies))

In [0]:
mean = {}
for j in range(1,Users):
  mean[j] = np.sum(Rating_matrix[j,:])/cnts[j];
  #print(mean)
  for i in  range(0,Total_Movies):
    if (Rating_matrix[j,i]!=0):
      rating_matrix[j,i] = Rating_matrix[j,i]
      rating_matrix[j,i]-=mean[j]

In [22]:
rating_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.36637931,  0.        , -0.36637931, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.63417569, -1.13417569, -1.13417569, ...,  0.        ,
         0.        ,  0.        ],
       [-0.27027027,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [0]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [0]:
user_similarity = fast_similarity(rating_matrix, kind='user')
item_similarity = fast_similarity(rating_matrix, kind='item')

In [27]:
item_similarity

array([[1.00000000e+00, 6.18601924e-02, 5.15510384e-02, ...,
        3.96416578e-10, 3.96416578e-10, 2.18679953e-10],
       [6.18601924e-02, 1.00000000e+00, 5.62521781e-02, ...,
        6.32544856e-10, 6.32544856e-10, 3.48938180e-10],
       [5.15510384e-02, 5.62521781e-02, 1.00000000e+00, ...,
        7.52578595e-10, 7.52578595e-10, 4.15153808e-10],
       ...,
       [3.96416578e-10, 6.32544856e-10, 7.52578595e-10, ...,
        1.00000000e+00, 1.00000000e+00, 1.30978907e-08],
       [3.96416578e-10, 6.32544856e-10, 7.52578595e-10, ...,
        1.00000000e+00, 1.00000000e+00, 1.30978907e-08],
       [2.18679953e-10, 3.48938180e-10, 4.15153808e-10, ...,
        1.30978907e-08, 1.30978907e-08, 1.00000000e+00]])

In [29]:
np.sum(rating_matrix*rating_matrix,axis = 0)

array([1.51091691e+02, 5.93419130e+01, 4.19218534e+01, ...,
       4.21168412e-02, 4.21168412e-02, 1.38401715e-01])

#Recommendation: Similar Items to a given item

In [62]:
#Longer method: Taking similarity with each item 
x = int(input("Enter the movieId to which you want to search similar items: "))
y = Id_Map[x]
Movie_column = rating_matrix[:,y].reshape((-1,1))
Similarity_list = []
Square_matrix = np.sqrt(np.sum(rating_matrix*rating_matrix,axis = 0))
for i in range(0,Total_Movies):
    Movie_i = rating_matrix[:,i].reshape(-1,1)
    if (Square_matrix[i]*Square_matrix[y]==0):
      similarity = -1
    else:
      similarity = (Movie_column.T).dot(Movie_i)/(Square_matrix[i]*Square_matrix[y])
    Similarity_list.append(similarity)
Similarity_list = np.array(Similarity_list).reshape(-1,1)
Final = sorted(range(len(Similarity_list)), key=lambda i: Similarity_list[i])[-4:]
print("Top 3 Recommended films which are similar to given film are: ")
for i in range(0,3):
    print(Movies.iloc[Final[2-i],1])

Enter the movieId to which you want to search similar items: 2
Top 3 Recommended films which are similar to given film are: 
Loser (1991)
Return from Witch Mountain (1978)
Bad Words (2013)


In [64]:
#From the item_similarity Matrix
Similarity_list = item_similarity[y,:]
Final = sorted(range(len(Similarity_list)), key=lambda i: Similarity_list[i])[-4:]
print("Top 3 Recommended films which are similar to given film are: ")
for i in range(0,3):
    print(Movies.iloc[Final[2-i],1])

Top 3 Recommended films which are similar to given film are: 
Loser (1991)
Return from Witch Mountain (1978)
Bad Words (2013)
